In [22]:
### Unify the notation for the different clinical variables across all centers and hospitals in MD Anderson samples ############

#  I import the necessary modules to work
import pandas as pd
import numpy as np
import re
import os
# I read the prefiltered file for MDA cohort
mda_path='/home/vant/TFM/MDA_bbdd_filtered1.xlsx'
mda=pd.read_excel(mda_path)
print(mda.head(5)) # I see the row with index 0 contains the subheaders
print("The length of MDA dataframe prefiltered is:",len(mda)) #53

   ID CNIO                                          HISTOLOGY  \
0  ID CNIO  1=serous, 2=mucinous, 3=endometrioid, 4=clear ...   
1     MDA1                                                  4   
2     MDA2                                                  3   
3     MDA3                                                  4   
4     MDA4                                                  3   

  AGE AT DIAGNOSIS                                        TUMOR GRADE  \
0            years  1=well differentiated, 2=moderately differenti...   
1               80                                                  3   
2               45                                                  3   
3               57                                                  2   
4               32                                                  1   

                                          FIGO STAGE  \
0  1=IA, 2=IB, 3=IC, 4=I(NOS), 5=IIA, 6=IIB, 7=II...   
1                                                  6   
2 

In [23]:
print(mda.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 19 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   ID CNIO                                                            53 non-null     object
 1   HISTOLOGY                                                          53 non-null     object
 2   AGE AT DIAGNOSIS                                                   53 non-null     object
 3   TUMOR GRADE                                                        53 non-null     object
 4   FIGO STAGE                                                         52 non-null     object
 5   FIGOa                                                              52 non-null     object
 6   FIGOL                                                              52 non-null     object
 7   Residual Disease AFTER SURGERY       

In [24]:
# If I want to eliminate the subheaders for next calculations
mda_no_subheaders=mda.drop([0]) #52 rows =52 samples
mda_no_subheaders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 1 to 52
Data columns (total 19 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   ID CNIO                                                            52 non-null     object
 1   HISTOLOGY                                                          52 non-null     object
 2   AGE AT DIAGNOSIS                                                   52 non-null     object
 3   TUMOR GRADE                                                        52 non-null     object
 4   FIGO STAGE                                                         51 non-null     object
 5   FIGOa                                                              51 non-null     object
 6   FIGOL                                                              51 non-null     object
 7   Residual Disease AFTER SURGERY       

In [25]:
# Firstly I rename the columns 
# Rename columns 
new_column_names = {
    'ID CNIO': 'ID_CNIO',
    'AGE AT DIAGNOSIS':'AGE',
    'TUMOR GRADE':'GRADE',
    'FIGO STAGE':'FIGO',
    'Residual Disease AFTER SURGERY': 'RESIDUAL',
    'RESIDUALsD':'RESIDUALa',
    'FIRST LINE or adjuvant TREATMENT': 'ADJUVANT_TREATMENT',
    'First line regimen':'TYPE_ADJUVANT',
    'MMR GERMLINE STATUS':'MMR STATUS',
    'DATE USED FOR OS':'DATE_OS',
    'FAMILIAL ANTECEDENTS OF  COLON, ENDOMETRIAL AND/OR OVARIAN CANCER':'FAMILIAL'
    
}
mda_no_subheaders.rename(columns=new_column_names,inplace=True)
mda_no_subheaders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 1 to 52
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID_CNIO                   52 non-null     object
 1   HISTOLOGY                 52 non-null     object
 2   AGE                       52 non-null     object
 3   GRADE                     52 non-null     object
 4   FIGO                      51 non-null     object
 5   FIGOa                     51 non-null     object
 6   FIGOL                     51 non-null     object
 7   RESIDUAL                  44 non-null     object
 8   RESIDUALa                 44 non-null     object
 9   ADJUVANT_TREATMENT        48 non-null     object
 10  TYPE_ADJUVANT             40 non-null     object
 11  MMR STATUS                52 non-null     object
 12  BRCA STATUS               51 non-null     object
 13  VITAL STATUS              52 non-null     object
 14  OS_CNIO                   51

In [26]:
# Transform coding in HISTOLOGY column to the final coding
# Actual coding: 1=serous, 2=mucinous, 3=endometrioid, 4=clear cell, 
# 5=mixed cell, 6=other specified epithelial ovarian cancer (e.g. Brenner), 7=undifferentiated epithelial, NA=don´t know
# New coding:0=endometroid;1=clear cells;2=mixed,3=others,NA=unknown'
def convert_histology(data1):
    if data1==3:
        return 0
    elif data1==4:
        return 1
    elif data1==1 or data1==2:
        return 3
    elif data1==5:
        return 2
    elif pd.isna(data1):
        return 'NA'
print(mda_no_subheaders['HISTOLOGY'].head(15))
mda_no_subheaders['HISTOLOGY']= mda_no_subheaders['HISTOLOGY'].apply(convert_histology)
print(mda_no_subheaders['HISTOLOGY'].head(15))

1     4
2     3
3     4
4     3
5     4
6     3
7     3
8     4
9     4
10    4
11    3
12    3
13    3
14    3
15    3
Name: HISTOLOGY, dtype: object
1     1
2     0
3     1
4     0
5     1
6     0
7     0
8     1
9     1
10    1
11    0
12    0
13    0
14    0
15    0
Name: HISTOLOGY, dtype: int64


In [27]:
# Transform coding in RESIDUAL column
# Actual:0=no macroscopic disease, 1=macroscopic disease <1 cm; 2=macroscopic disease >1;  NA=macroscopic disease, size unknown
# Now we mantain but 1=macroscopic disease<=1 cm. We don´t have to modify this o RESIDUALsD column but we have to change the values 
# New: 0=no macroscopic disease, 1=macroscopic disease <=1 cm; 2=macroscopic disease >1; 3=macroscopic disease, size unknown; NA=unknown
print(mda_no_subheaders['RESIDUAL'].head(15))

# Replace empty cells by 'NA'. We wait at the moment. Maybe in next step we convert all empty cells to NA
mda_no_subheaders['RESIDUAL'] = mda_no_subheaders['RESIDUAL'].fillna('NA')
print(mda_no_subheaders['RESIDUAL'].head(15))


1       3
2       3
3       3
4       0
5       0
6       0
7       3
8       3
9       3
10    NaN
11      3
12      0
13    NaN
14      0
15    NaN
Name: RESIDUAL, dtype: object
1      3
2      3
3      3
4      0
5      0
6      0
7      3
8      3
9      3
10    NA
11     3
12     0
13    NA
14     0
15    NA
Name: RESIDUAL, dtype: object


In [28]:
###### AT FIRST, I KEEP ND CELLS AS SUCH TO DIFFERENTIATE THEM FROM WHERE THERE IS NO DATA (NA) ###############

# Transform coding in ADJUVANT_TREATMENT column to the final coding.
# The only thing I need to do is change the 'ND' value to 'NA' in order to keep all values consistent.
# There are also empty cells, but we don’t know the exact differences between ND and empty cells,
# so we will standardize them both as 'NA' to maintain consistency:
# print(mda_no_subheaders['ADJUVANT_TREATMENT'])

# Replace 'ND' with 'NA'
# Strip any extra spaces from the 'ADJUVANT_TREATMENT' column if the value is a string.
# This step is to ensure that the text does not have leading or trailing spaces before replacing.
# mda_no_subheaders['ADJUVANT_TREATMENT'] = mda_no_subheaders['ADJUVANT_TREATMENT'].apply(lambda x: x.strip() if isinstance(x, str) else x)

# Replace all occurrences of 'ND' with 'NA' in the column.
# mda_no_subheaders['ADJUVANT_TREATMENT'] = mda_no_subheaders['ADJUVANT_TREATMENT'].replace('ND', 'NA')

# Uncomment the next line to check the transformation in the column after the replacement
# print(mda_no_subheaders['ADJUVANT_TREATMENT'])

In [29]:
# Transform coding in TYPE_ADJUVANT
# Actual: 0=Carbo-paclitaxel 1=Cis-paclitaxel iv,2=Cis-pacli IP,3=Carbo monoterapia 4=Carbo-Taxol-Beva ,5=otro.
# New: 0=Carbo-paclitaxel 1=Cis-paclitaxel, 2=Carbo-monotherapy, 3=Carbo-Taxol-Beva ;4=other. 
# I create a dictionary to specify the changes
# Function to modify TYPE_ADJUVANT:
def adjust_type_adjuvant(row):
    value = row['TYPE_ADJUVANT']
    if value == '0(neoad)+ Caelyx + Beva(2ª línea)+ …':
        return 4, value
    if isinstance(value, str) and ':' in value:
        # Extraer el número antes del ':' y la cadena después del ':'
        number_part = int(value.split(':')[0].strip())
        string_part = value.split(':')[1].strip()
        return number_part - 1, string_part
    elif value == '1' or value == 1 or value == '2' or value == 2:
        return 1, 'NA'
    elif value == '3' or value == 3:
        return 2, 'NA'
    elif value == '4' or value == 4:
        return 3, 'NA'
    elif value==5:
        return 4, 'NA'
    else:
        return value, 'NA'

print(mda_no_subheaders['TYPE_ADJUVANT'].head(35))
# Apply
mda_no_subheaders[['TYPE_ADJUVANT', 'OTHER_ADJ_TREAT']] = mda_no_subheaders.apply(adjust_type_adjuvant, axis=1, result_type='expand')
print(mda_no_subheaders[['TYPE_ADJUVANT', 'OTHER_ADJ_TREAT']].head(60))
# Replace ND values and ND (MDA only surgery) by NA
mda_no_subheaders['TYPE_ADJUVANT'] = mda_no_subheaders['TYPE_ADJUVANT'].replace('ND', 'NA')
mda_no_subheaders['TYPE_ADJUVANT'] = mda_no_subheaders['TYPE_ADJUVANT'].replace('ND (MDA only surgery)', 'NA')
print(mda_no_subheaders[['TYPE_ADJUVANT', 'OTHER_ADJ_TREAT']].head(60))

1                        ND
2                       NaN
3                        ND
4                        ND
5                         0
6                       NaN
7                         0
8                       NaN
9                         0
10                      NaN
11                        0
12                      NaN
13                      NaN
14                      NaN
15                        0
16                        0
17                        0
18                      NaN
19                        0
20                        0
21    ND (MDA only surgery)
22                        0
23    ND (MDA only surgery)
24                      NaN
25                      NaN
26                        0
27                        0
28           5: EC PENOLOPE
29                        0
30                        0
31                      NaN
32                        2
33                       0?
34                        0
35                        4
Name: TYPE_ADJUVANT,

In [30]:
# Recoding the MMMR STATUS and BRCA STATUS columns
# In these columns the coding is:  0=unknown; 1=mutated; 2=unmutated;
# and now MMR STATUS/BRCA STATUS: 0=not studied,1=studied,without mutation,2=mutation in MMR genes/BRCA1/2,3=other genes mutated, NA= no data
# Having into account that NA can be LYNCH/BRCA in BRCA STATUS/MMR STATUS
def calculate_mmr_status(data4):
    if data4 == 0:
        return 0
    elif data4 == 1:
        return 2
    elif data4 == 2:
        return 1
    elif pd.isna(data4):
        return data4
def calculate_brca_status(data5):
    if data5 == 0:
        return 0
    elif data5 == 1:
        return 2
    elif data5 == 2:
        return 1
    elif pd.isna(data5):
        return data5
columns_to_print=['ID_CNIO','MMR STATUS','BRCA STATUS']
print(mda_no_subheaders[columns_to_print].head(40))
mda_no_subheaders['MMR STATUS']=mda_no_subheaders['MMR STATUS'].apply(calculate_mmr_status)
mda_no_subheaders['BRCA STATUS']=mda_no_subheaders['BRCA STATUS'].apply(calculate_brca_status)
print(mda_no_subheaders[columns_to_print].head(40))

   ID_CNIO MMR STATUS BRCA STATUS
1     MDA1          0           0
2     MDA2          0           0
3     MDA3          0           0
4     MDA4          0           2
5     MDA5          0           0
6     MDA6          0           0
7     MDA7          0           0
8     MDA8          0           0
9     MDA9          0           0
10   MDA10          0           0
11   MDA11          0           2
12   MDA12          0           0
13   MDA13          0           0
14   MDA14          0           0
15   MDA15          0           1
16   MDA16          1           2
17   MDA17          0           0
18   MDA18          0           0
19   MDA19          0           2
20   MDA20          0           0
21   MDA21          0           0
22   MDA22          0           0
23   MDA23          0           0
24   MDA24          0           0
25   MDA25          0           0
26   MDA26          0           0
27   MDA27          0           2
28   MDA28          0           0
29   MDA29    

In [31]:
# I see that in the column 'DATE FOR OS' some data are lacking for OS, but in this column Diagnosis is the value
# I change this for the value 'Lacking diagnosis/death_lastv date'
mda_no_subheaders['DATE_OS'] = mda_no_subheaders.apply(
    lambda row: 'LD' if pd.isna(row['OS_CNIO']) else row['DATE_OS'],
    axis=1
)

print("\nDataFrame updated:")
print(mda_no_subheaders[['OS_CNIO','DATE_OS']])


DataFrame updated:
   OS_CNIO DATE_OS
1      NaN      LD
2     3202       D
3      320       D
4     6715       D
5     4869       D
6     4740       D
7     4816       D
8      186       D
9     2222       D
10     266       D
11    3109       D
12    3636       D
13    2206       D
14       2       D
15      -2       D
16    4875       D
17     184       D
18    3286       D
19    2885       D
20    2850       D
21    1617       D
22    2731       D
23      79       D
24    2928       D
25    2804       D
26    2785       D
27    1687       D
28     575       D
29     105       D
30    2211       D
31    2057       D
32    2279       D
33     187       D
34    3122       D
35     127       D
36     204       D
37    1653       D
38    1660       D
39    1300       D
40    1171       D
41     393       D
42      28       D
43     426       D
44     402       D
45     314       D
46     757       D
47     529       D
48     468       D
49     499       D
50     374       D
51     308 

In [32]:
print(mda_no_subheaders[['VITAL STATUS','ADJUVANT_TREATMENT']])
mda_no_subheaders['VITAL STATUS']=mda_no_subheaders['VITAL STATUS'].replace('ND','NA')
mda_no_subheaders['ADJUVANT_TREATMENT']=mda_no_subheaders['ADJUVANT_TREATMENT'].replace('ND','NA')
mda_no_subheaders['FAMILIAL']=mda_no_subheaders['FAMILIAL'].replace('ND','NA')
print(mda_no_subheaders[['VITAL STATUS','ADJUVANT_TREATMENT']])

   VITAL STATUS ADJUVANT_TREATMENT
1             1                 ND
2             0                  1
3             1                 ND
4             0                  1
5             0                  1
6             0                  0
7             1                  1
8             0                NaN
9             0                  1
10            1                NaN
11            0                  1
12            0                  0
13            1                NaN
14            0                NaN
15            0                 ND
16            0                  1
17            1                  1
18            0                  0
19            0                  1
20            0                  1
21            0                  1
22            0                  1
23            0                 ND
24            0                  0
25            0                  0
26            0                  1
27            0                  1
28            1     

In [33]:
mda_no_subheaders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 1 to 52
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID_CNIO                   52 non-null     object 
 1   HISTOLOGY                 52 non-null     int64  
 2   AGE                       52 non-null     object 
 3   GRADE                     52 non-null     object 
 4   FIGO                      51 non-null     object 
 5   FIGOa                     51 non-null     object 
 6   FIGOL                     51 non-null     object 
 7   RESIDUAL                  52 non-null     object 
 8   RESIDUALa                 44 non-null     object 
 9   ADJUVANT_TREATMENT        48 non-null     object 
 10  TYPE_ADJUVANT             40 non-null     object 
 11  MMR STATUS                52 non-null     int64  
 12  BRCA STATUS               51 non-null     float64
 13  VITAL STATUS              52 non-null     object 
 14  OS_CNIO     

In [34]:
# Order the columns in the definitive order
# New order
new_order = [
    'ID_CNIO', 'HISTOLOGY', 'AGE', 'GRADE', 'FIGO',
    'FIGOL', 'FIGOa', 'ADJUVANT_TREATMENT', 'TYPE_ADJUVANT','OTHER_ADJ_TREAT',
    'RESIDUAL', 'RESIDUALa', 'OS_CNIO', 'VITAL STATUS', 'MMR STATUS', 'BRCA STATUS','FAMILIAL',
    'DATE_OS','PARTIAL DATE DEATH_LASTv','PARTIAL DATE DIAGNOSIS'
]

# Reorder
mda_no_subheaders = mda_no_subheaders[new_order]
print(mda_no_subheaders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 1 to 52
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID_CNIO                   52 non-null     object 
 1   HISTOLOGY                 52 non-null     int64  
 2   AGE                       52 non-null     object 
 3   GRADE                     52 non-null     object 
 4   FIGO                      51 non-null     object 
 5   FIGOL                     51 non-null     object 
 6   FIGOa                     51 non-null     object 
 7   ADJUVANT_TREATMENT        48 non-null     object 
 8   TYPE_ADJUVANT             40 non-null     object 
 9   OTHER_ADJ_TREAT           52 non-null     object 
 10  RESIDUAL                  52 non-null     object 
 11  RESIDUALa                 44 non-null     object 
 12  OS_CNIO                   51 non-null     object 
 13  VITAL STATUS              52 non-null     object 
 14  MMR STATUS  

In [35]:
# Create row of subheaders/coding row (row below the description row)
subheaders = ['LP: ID for samples from La Paz Hospital; OVE: ID for samples from Virgen del Rocio Hospital; MDA: ID for samples from MD Anderson Cancer Center  Hospital ; RVB: ID for samples from Red Valenciana de Biobancos',
             '0=endometroid; 1=clear cells; 2=mixed; 3=others; NA=unknown', '2 digits; unit is YEARS',
             '1=well differentiated; 2=moderately differentiated; 3=poorly differentiated; NA=unknown',
             '1=IA; 2=IB; 3=IC; 4=I(NOS); 5=IIA; 6=IIB; 8=II(NOS); 9=IIIA; 10=IIIB; 11=IIIC; 12=III(NOS); 13=IV; NA=unknown',
             '0=localized (I,II); 1=advanced(III,IV); NA=unknown','1= IA, IB, IC, I(NOS); 2=IIA, IIB, II (NOS); 3=IIIA, IIIB, IIIC, III(NOS); 4= IV; NA=unknown',
             '0=NO; 1=YES; NA=unknown',
             '0=Carbo-paclitaxel; 1=Cis-paclitaxel; 2=Carbo-monotherapy; 3=Carbo-Taxol-Beva; 4=other; NA=unknown','Drugs used as adjuvant treatment',
             '0=no macroscopic disease; 1=macroscopic disease<=1; 2=macroscopic disease>1; 3=macroscopic disease,unknown size; NA=unknown',
             '0=No residual disease; 1=Yes residual disease; NA=unknown','Unit is DAYS',
             '0=alive; 1=dead; LF:lost to follow-up; NA=unknown','0=not studied; 1=studied,without mutation; 2=mutation in MMR genes; 3=other genes mutated; NA=unknown',
             '0=not studied; 1=studied,without mutation; 2=mutation in BRCA1/2; 3=other genes mutated;NA=unknown','0=NO; 1=YES; NA=unknown',
             'D=diagnosis date used for OS; S=surgery date used for OS; LD:Lacking diagnosis/surgery/death_lastv date;NA=unknown',
             'YES= partial date xx/xx/y or xx/m/y; NO= Complete date or NA','YES= partial date xx/xx/y or xx/m/y; NO= Complete date or NA'
             ]
df_subheaders = pd.DataFrame([subheaders], columns=mda_no_subheaders.columns)

# Concatenate the row of subheaders below the original DataFrame
df_with_subheaders = pd.concat([df_subheaders, mda_no_subheaders], ignore_index=True)
print("\nDataFrame con subheaders añadidos:")
print(df_with_subheaders)


DataFrame con subheaders añadidos:
                                              ID_CNIO  \
0   LP: ID for samples from La Paz Hospital; OVE: ...   
1                                                MDA1   
2                                                MDA2   
3                                                MDA3   
4                                                MDA4   
5                                                MDA5   
6                                                MDA6   
7                                                MDA7   
8                                                MDA8   
9                                                MDA9   
10                                              MDA10   
11                                              MDA11   
12                                              MDA12   
13                                              MDA13   
14                                              MDA14   
15                                              MDA1

In [36]:
# Row with descriptions- I want it in the first row below the headers (lista o diccionario)
descriptions = {
    'ID_CNIO': 'Unique CNIO tumor identifier',
    'HISTOLOGY': 'Tumor histology/Tumor type',
    'AGE': 'Age at diagnosis. Calculated as (date of diagnosis – date of birth). When not provided, date of surgery was used instead of date of diagnosis (OVE series); a tag column was added to indicate this: “DATE_AGE”.',
    'GRADE':'Tumor differentiation grade',
    'FIGO':'FIGO tumor stage',
    'FIGOL':'FIGO tumor stage according to cancer spread (localized or advanced)',
    'FIGOa':' Aggregated tumor stage',
    'ADJUVANT_TREATMENT':'Information about whether the patient received chemotherapy',
    'TYPE_ADJUVANT':'Type of adjuvant treatment received. Carboplatino+taxol is the standard',
    'OTHER_ADJ_TREAT':'Other type of adjuvant treatment received by the patient different from the standard, carboplatino+taxol',
    'RESIDUAL':'Residual disease after surgery',
    'RESIDUALa':'Aggregated categories of residual disease after surgery',
    'OS_CNIO':'Overall survival calculated by CNIO from original records as (date of death-date of diagnosis) or (date last known to be alive-date of diagnosis). When not provided, date of surgery was used instead of date of diagnosis. In this last case it is indicated in the tag column DATE USED FOR OS',
    'VITAL STATUS':'Vital status at last followup',
    'MMR STATUS':'Information about the GERMILINE mutations in genes related to mismatch repair pathway',
    'BRCA STATUS':'Information about GERMILINE mutation in BRCA1/2 genes',
    'FAMILIAL':'Familial antecedents of colon,endometrial and/or ovarian cancer. In OVE series this information refers to possible hereditary conditions according to the clinician',
    'DATE_OS':'Tag column to indicate how overall survival was calculated, using date of diagnosis or date of surgery (OVE series)',
    'PARTIAL DATE DEATH_LASTv':'Tag column to indicate how overall survival was calculated, using a partial or a complete date of last visit. If a partial date is used this suggests that the estimation of OS is not exact. A partial date is coded as XX/XX/year or XX/month/year and to make it possible the calculation these are transformed into 01/06/year or 01/month/year respectively',
    'PARTIAL DATE DIAGNOSIS':'Tag column to indicate how overall survival was calculated, using a partial or a complete date of diagnosis. If a partial date is used this suggests that the estimation of OS is not exact. A partial date is coded as XX/XX/year or XX/month/year and to make it possible the calculation these are transformed into 01/06/year or 01/month/year respectively' 
}

# Convert the dictionary into a DataFrame
desc_df = pd.DataFrame(descriptions, index=[0])

# Concatenate
df_final = pd.concat([desc_df, df_with_subheaders]).reset_index(drop=True)

# Show
print(df_final)  

                                              ID_CNIO  \
0                        Unique CNIO tumor identifier   
1   LP: ID for samples from La Paz Hospital; OVE: ...   
2                                                MDA1   
3                                                MDA2   
4                                                MDA3   
5                                                MDA4   
6                                                MDA5   
7                                                MDA6   
8                                                MDA7   
9                                                MDA8   
10                                               MDA9   
11                                              MDA10   
12                                              MDA11   
13                                              MDA12   
14                                              MDA13   
15                                              MDA14   
16                             

In [37]:
# Order the samples
import re

# Separate the first two rows (subheaders and coding)
header_rows = df_final.iloc[:2]

# The rest of the DataFrame (starting from row 2)
data_rows = df_final.iloc[2:].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Extract the number after "MDA" only for the data rows
data_rows.loc[:, 'ID_number'] = data_rows['ID_CNIO'].str.extract(r'MDA(\d+)')

# Convert the extracted values to numbers, ignoring errors (if the extraction fails)
data_rows.loc[:, 'ID_number'] = pd.to_numeric(data_rows['ID_number'], errors='coerce')

# Filter rows where 'ID_number' is not NaN (keep only valid rows with extracted numbers)
data_rows = data_rows[data_rows['ID_number'].notna()]

# Sort the DataFrame by the extracted number ('ID_number')
data_rows = data_rows.sort_values(by='ID_number').reset_index(drop=True)

# Drop the auxiliary column ('ID_number') if it is no longer needed
data_rows = data_rows.drop(columns=['ID_number'])

# Recombine the header rows with the sorted data rows to create the final DataFrame
df_final_sorted = pd.concat([header_rows, data_rows], ignore_index=True)

In [38]:
# Add original identifiers columns from the file passed by Maria
# Read
identificadores_df = pd.read_excel("/home/vant/TFM/Identificadores OVE_LP_MDA_RVB_serie completa.xlsx",sheet_name=0)
print(identificadores_df.head())
print(identificadores_df.columns) # I realise there are additional spaces in ID CNIO header
identificadores_df.columns = identificadores_df.columns.str.strip()
print(identificadores_df.columns)

                 ID CNIO                                      ORIGINAL ID_AP  \
0  unique identifier CNIO  unique identifier from hospital, "Anatomía Pat...   
1                   07T25                                         8175-04/A4   
2                   08T94                          99B5119-8 (F.H. ALCORCÓN)   
3                   08T96                         04B0008135 (F.H. ALCORCÓN)   
4                  09T134                      B781787  (H,Gregorio Marañon)   

                               ORIGINAL ID_NHC_BBANK  \
0  unique identifier from Hospital (LP, OVE, MDA ...   
1                                       BTCNIO06/111   
2                                                NaN   
3                                                NaN   
4                                            B781787   

                   SUBTIPO HISTOLÓGICO  \
0  CC=clear cell ovarian carcinoma; E=   
1                                   CC   
2                                    E   
3             

In [39]:
# Add original identifiers columns from the file passed by Maria
print(identificadores_df.head())
# Rename 'ID CNIO' to match with 'ID_CNIO' of the DataFrame
identificadores_df.rename(columns={'ID CNIO': 'ID_CNIO'}, inplace=True)

# Merge using the common column
df_final2 = df_final_sorted.merge(identificadores_df[['ID_CNIO', 'ORIGINAL ID_AP', 'ORIGINAL ID_NHC_BBANK']], 
                    on='ID_CNIO', 
                    how='left')

# Check
print(df_final2.head())

                  ID CNIO                                     ORIGINAL ID_AP  \
0  unique identifier CNIO  unique identifier from hospital, "Anatomía Pat...   
1                   07T25                                         8175-04/A4   
2                   08T94                          99B5119-8 (F.H. ALCORCÓN)   
3                   08T96                         04B0008135 (F.H. ALCORCÓN)   
4                  09T134                      B781787  (H,Gregorio Marañon)   

                               ORIGINAL ID_NHC_BBANK  \
0  unique identifier from Hospital (LP, OVE, MDA ...   
1                                       BTCNIO06/111   
2                                                NaN   
3                                                NaN   
4                                            B781787   

                   SUBTIPO HISTOLÓGICO  \
0  CC=clear cell ovarian carcinoma; E=   
1                                   CC   
2                                    E   
3             

In [40]:
# Add description and coding in rows with index 0 and 1
# Rename'ORIGINAL ID_AP' to 'ID_ORIGINAL' and ORIGINAL ID_NHC_BBANK to 'ID_ORIGINAL_NHC_BBANK'
df_final2.rename(columns={'ORIGINAL ID_AP': 'ID_ORIGINAL'}, inplace=True)
df_final2.rename(columns={'ORIGINAL ID_NHC_BBANK': 'ID_ORIGINAL_NHC_BBANK'}, inplace=True)
# Assign values to rows 0 and 1 
df_final2.loc[0, 'ID_ORIGINAL'] = 'Unique identifier from the hospital to the patient/sample'
df_final2.loc[1, 'ID_ORIGINAL'] = 'Pathology code'
# Assign values to rows 0 and 1
df_final2.loc[0, 'ID_ORIGINAL_NHC_BBANK'] = 'Unique identifier from Hospital (LP, OVE, MDA series; NHC= number of clinical history) or Biobank (RVB series) to the patient/sample'
df_final2.loc[1, 'ID_ORIGINAL_NHC_BBANK'] = 'Unique alphanumeric code from the hospital or biobank.'

In [41]:
df_final2.to_excel('/home/vant/TFM/MDA_final.xlsx',index=False)
print("File generated")

File generated
